# Planet Order and Delivery to GEE

*Notebook based on Planetlabs example notebooks for [GEE Delivery](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/gee-integration/gee-integration.ipynb) and the [Data API Introduction for Python](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/planet_data_api_introduction.ipynb)*

- *click [here](https://developers.planet.com/docs/apis/data/) for general info on the Planet API*
- *click [here](https://developers.planet.com/docs/data/psscene/) for more info on the specifications of the PlanetScope Satellite products and the general order process*

**Prerequisites:**
- Planet's Python SDK 2.0 installed and initialized in your environment. Click [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/get-started/quick-start-guide/) for more info and instructions.
- An AOI : `AOI`
- A GEE project with EE API enabled `braided-rivers-ee`
- A pre-existing GEE ImageCollection `planet_collection`
- An account with a download quota

## Set-up

In [1]:
import os
import json
import requests
import geojsonio
import time

import planet

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

# include here your own Planet API Key (find it under "My Account"->"My Settings"
PLANET_API_KEY = 'ef5d387992604ce5ab17734fae36990d'

In [2]:
# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the quick search endpoint url
quick_url = "{}/quick-search".format(URL)

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (PLANET_API_KEY, "")

# Make a GET request to the Planet Data API - if the response is "200" the authetification process was succesful!
res = session.get(URL)
print(res)

<Response [200]>


### GEE Set-up

In [3]:
# Google Earth Engine configuration
# Define cloud delivery location:
cloud_config = planet.order_request.google_earth_engine(
    project='braided-rivers-ee', collection='planet_eygue7_2021')

# Order delivery configuration:
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

In [4]:
# Function to create and deliver the order
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

## Definition of AOI
--> needs to be one feature, so no feature collection is allowed! For that reason, the different features are merged into one single Multipolygon-object.

Options: Either delineate features manually via https://geojson.io/ or merge together and make sure that the merged object represents a Feature and not a FeatureCollection

In [5]:
# AOI defined as GeoJSON Multipolygon 

AOI = {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                4.91588943510856,
                44.211980701554474
              ],
              [
                4.919121104676019,
                44.20944095623606
              ],
              [
                4.91782520007276,
                44.2090870698574
              ],
              [
                4.917265335801108,
                44.20906533965949
              ],
              [
                4.916863957485318,
                44.2087893505205
              ],
              [
                4.916624011016038,
                44.20867849898615
              ],
              [
                4.916466955475128,
                44.2084316942382
              ],
              [
                4.916082174189892,
                44.20834173908083
              ],
              [
                4.915159398394788,
                44.20746633459301
              ],
              [
                4.914384948290356,
                44.20735644015716
              ],
              [
                4.913334225811278,
                44.20669377205252
              ],
              [
                4.912064040486246,
                44.20530220865854
              ],
              [
                4.910489495082756,
                44.204873245532625
              ],
              [
                4.909456778403319,
                44.20407029846575
              ],
              [
                4.908477366850409,
                44.20372633032983
              ],
              [
                4.907598868042738,
                44.203642142349025
              ],
              [
                4.907353478653067,
                44.20269830325958
              ],
              [
                4.903788349562426,
                44.20378323559256
              ],
              [
                4.905877227401128,
                44.205551796476904
              ],
              [
                4.907243672660227,
                44.20710428789758
              ],
              [
                4.909992274706736,
                44.20942913750944
              ],
              [
                4.911682547034496,
                44.20995930400439
              ],
              [
                4.91351085131234,
                44.21026575247416
              ],
              [
                4.915128553182849,
                44.210926694890105
              ],
              [
                4.91588943510856,
                44.211980701554474
              ]
            ]
          ]
        ]
      }

## Setting up image filter

In [7]:
# Daterange filter
date_filter = {
    "type": "DateRangeFilter", # Type of filter -> Date Range
    "field_name": "acquired", # The field to filter on: "acquired" -> Date on which the "image was taken"
    "config": {
        "gte": "2021-01-01T00:00:00.000Z", # "gte" -> Greater than or equal to
        "lte": "2021-12-31T23:59:59.999Z" # "lte" -> Lower than or equal to
    }
}

# geometry filter on defined AOI
geom_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI
}

# quality assurance
quality_filter = {
        "type": "StringInFilter",
        "field_name": "quality_category",
        "config": ["standard"]
      }

# cloud pre-filtering
cloud_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# Setup an "AND" logical filter
and_filter = {
    "type": "AndFilter",
    "config": [date_filter, geom_filter, quality_filter, cloud_filter]
}



# Print the logical filter
# p(and_filter)

## Requesting image IDs fitting the filter

In [8]:
# Specify the sensors/satellites or "item types" to include in our results
item_types = ["PSScene"]

# Construct the request.
request = {
    "item_types" : item_types,
    "filter" : and_filter
}

# Send the POST request to the API quick search endpoint
res = session.post(quick_url, json=request)
geojson = res.json()

# Initialize the item_ids list to collect all item ids
item_ids = []

# Collect item IDs from current page
features = geojson["features"]
current_page_item_ids = [f['id'] for f in features]
item_ids.extend(current_page_item_ids)

## Looping over several pages to extract >250 items

# Start the loop to paginate through all available pages
while current_page_item_ids:

    # Check if there's a next page, if not, break the loop
    next_url = geojson["_links"]["_next"] # pr: ["_next"]
    
    # Update the request URL for the next iteration
    res_next = session.get(next_url)
    geojson = res_next.json()
    
    # Collect item IDs from next page
    features_next = geojson["features"]
    current_page_item_ids = [f['id'] for f in features_next]
    item_ids.extend(current_page_item_ids)

# Print the total number of item IDs collected
print(len(item_ids), item_ids)

317 ['20211230_102803_88_2254', '20211217_125650_0f02', '20211214_094258_06_242b', '20211213_110617_21_1057', '20211213_110615_71_1057', '20211206_094201_90_242b', '20211203_110755_74_105d', '20211203_110754_24_105d', '20211202_094411_78_2436', '20211130_103211_21_2405', '20211120_093955_41_2423', '20211120_093600_42_1063', '20211114_100746_1005', '20211106_094132_21_241f', '20211106_094051_99_2451', '20211105_094152_87_2455', '20211102_094304_31_220b', '20211102_093814_72_106c', '20211028_094313_66_2429', '20211028_094311_36_2429', '20211027_103220_12_2274', '20211027_102950_20_2416', '20211026_093752_76_106c', '20211026_103000_82_2406', '20211025_103219_69_2407', '20211025_095210_94_2251', '20211025_095213_23_2251', '20211024_093909_84_2420', '20211023_101649_88_105c', '20211023_103036_26_2413', '20211023_094226_98_2448', '20211021_110534_66_105a', '20211018_094345_98_2460', '20211018_094343_68_2460', '20211017_103242_74_2405', '20211014_093946_87_2447', '20211014_093944_57_2447', '2

### include interval of 5 days for image selection

In [9]:
from datetime import datetime, timedelta

image_ids = item_ids

def filter_images_by_interval(image_ids, interval_days):
    # Convert image IDs to datetime objects and sort
    dates_images = sorted([
        (datetime.strptime(image_id.split('_')[0], '%Y%m%d'), image_id)
        for image_id in image_ids
    ], key=lambda x: x[0])

    # Initialize variables
    filtered_image_ids = []
    last_selected_date = None

    for date_image in dates_images:
        date, image_id = date_image

        # If this is the first image or the date is at least interval_days after the last selected date, select it
        if last_selected_date is None or date >= last_selected_date + timedelta(days=interval_days):
            last_selected_date = date
            # Find and include all images for the selected date
            filtered_image_ids.extend([
                img_id for img_date, img_id in dates_images if img_date == date
            ])
            
    return filtered_image_ids

# Call the function
filtered_image_ids = filter_images_by_interval(image_ids, interval_days=5)

# Print the selected image IDs
# len(filtered_image_ids)
print(len(filtered_image_ids), filtered_image_ids)

113 ['20210111_104836_03_1058', '20210116_101256_0f17', '20210116_101255_0f17', '20210124_094526_77_2212', '20210131_101241_1014', '20210131_101240_1014', '20210208_101333_1032', '20210213_103429_48_2401', '20210213_103427_22_2401', '20210213_095227_81_2271', '20210220_103625_63_2406', '20210226_103709_12_2254', '20210226_103706_76_2254', '20210226_103945_95_2405', '20210226_103943_68_2405', '20210305_094714_07_242a', '20210305_100953_1034', '20210310_103839_27_2412', '20210310_103836_92_2412', '20210310_105247_70_1057', '20210315_094334_89_2449', '20210315_101334_1003', '20210320_094309_80_2421', '20210320_103638_43_2412', '20210320_103458_08_2406', '20210320_103455_80_2406', '20210327_103518_58_225b', '20210401_094758_24_2434', '20210401_094535_31_2460', '20210406_101356_1010', '20210406_100925_1032', '20210406_100926_1032', '20210412_094328_09_2420', '20210418_103638_36_241c', '20210423_094304_99_242d', '20210423_094302_66_242d', '20210423_094522_17_106c', '20210423_094520_14_106c',

## Building the order request

In [13]:
# Product description for the order request, including image_ids from filter
data_products = [
    planet.order_request.product(item_ids=filtered_image_ids,
                                 product_bundle='analytic_8b_sr_udm2', # analytic surface reflectance 8-band multispectral band
                                 item_type='PSScene')
]


# Clip images to the AOI's perimeter and harmonize the data with Dove Classic
tools = [
    planet.order_request.clip_tool(AOI),
    planet.order_request.harmonize_tool('Sentinel-2')
]

# Build the order request
planet_order = planet.order_request.build_request(name='2021_eygue7_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(planet_order)

{'name': '2021_eygue7_order', 'products': [{'item_ids': ['20210111_104836_03_1058', '20210116_101256_0f17', '20210116_101255_0f17', '20210124_094526_77_2212', '20210131_101241_1014', '20210131_101240_1014', '20210208_101333_1032', '20210213_103429_48_2401', '20210213_103427_22_2401', '20210213_095227_81_2271', '20210220_103625_63_2406', '20210226_103709_12_2254', '20210226_103706_76_2254', '20210226_103945_95_2405', '20210226_103943_68_2405', '20210305_094714_07_242a', '20210305_100953_1034', '20210310_103839_27_2412', '20210310_103836_92_2412', '20210310_105247_70_1057', '20210315_094334_89_2449', '20210315_101334_1003', '20210320_094309_80_2421', '20210320_103638_43_2412', '20210320_103458_08_2406', '20210320_103455_80_2406', '20210327_103518_58_225b', '20210401_094758_24_2434', '20210401_094535_31_2460', '20210406_101356_1010', '20210406_100925_1032', '20210406_100926_1032', '20210412_094328_09_2420', '20210418_103638_36_241c', '20210423_094304_99_242d', '20210423_094302_66_242d', '

## Placing the Order for Delivery to GEE

In [14]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(planet_order, client)

00:02 - order  - state: creating


BadQuery: {"field":{"Details":[{"message":"No access to assets: PSScene/20210111_104836_03_1058/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210116_101256_0f17/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210116_101255_0f17/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210131_101241_1014/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210131_101240_1014/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210208_101333_1032/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210305_100953_1034/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210310_105247_70_1057/[ortho_analytic_8b_sr ortho_analytic_8b_xml]"},{"message":"No access to assets: PSScene/20210315_101334_1003/[ortho_analytic_8b_sr ortho_a

In [25]:
print(order_details)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/f59739b0-4e62-46c6-96d2-281b6dba6909'}, 'created_on': '2024-02-13T13:53:31.726Z', 'delivery': {'google_earth_engine': {'collection': 'planet_collection', 'project': 'braided-rivers-ee'}}, 'error_hints': [], 'id': 'f59739b0-4e62-46c6-96d2-281b6dba6909', 'last_message': 'Quota check failed - Over quota ', 'last_modified': '2024-02-13T13:53:53.861Z', 'name': 'test_order', 'products': [{'item_ids': ['20230601_093817_62_24b5', '20230601_093815_33_24b5', '20230601_100902_05_2490', '20230606_094116_16_24bb', '20230606_093533_58_24b4', '20230606_093531_24_24b4', '20230606_093254_08_242d', '20230606_093256_21_242d', '20230606_093251_94_242d', '20230606_093300_48_242d', '20230606_093258_35_242d', '20230606_101224_93_2488', '20230611_093837_30_24c9', '20230611_093841_90_24c9', '20230611_093844_21_24c9', '20230611_093530_85_2430', '20230611_093522_48_2430', '20230611_093528_76_2430', '20230611_093220_08_2427', '20230611_093213_81_2

## AOI

In [6]:
# AOI defined as GeoJSON Multipolygon 

AOI = {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                4.91588943510856,
                44.211980701554474
              ],
              [
                4.919121104676019,
                44.20944095623606
              ],
              [
                4.91782520007276,
                44.2090870698574
              ],
              [
                4.917265335801108,
                44.20906533965949
              ],
              [
                4.916863957485318,
                44.2087893505205
              ],
              [
                4.916624011016038,
                44.20867849898615
              ],
              [
                4.916466955475128,
                44.2084316942382
              ],
              [
                4.916082174189892,
                44.20834173908083
              ],
              [
                4.915159398394788,
                44.20746633459301
              ],
              [
                4.914384948290356,
                44.20735644015716
              ],
              [
                4.913334225811278,
                44.20669377205252
              ],
              [
                4.912064040486246,
                44.20530220865854
              ],
              [
                4.910489495082756,
                44.204873245532625
              ],
              [
                4.909456778403319,
                44.20407029846575
              ],
              [
                4.908477366850409,
                44.20372633032983
              ],
              [
                4.907598868042738,
                44.203642142349025
              ],
              [
                4.907353478653067,
                44.20269830325958
              ],
              [
                4.903788349562426,
                44.20378323559256
              ],
              [
                4.905877227401128,
                44.205551796476904
              ],
              [
                4.907243672660227,
                44.20710428789758
              ],
              [
                4.909992274706736,
                44.20942913750944
              ],
              [
                4.911682547034496,
                44.20995930400439
              ],
              [
                4.91351085131234,
                44.21026575247416
              ],
              [
                4.915128553182849,
                44.210926694890105
              ],
              [
                4.91588943510856,
                44.211980701554474
              ]
            ]
          ]
        ]
      }